In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import ImageGrid
from dotenv import load_dotenv

import os
import glob
import pandas as pd
import json

from db_tools import get_db

In [2]:
model = 'bruss'

In [3]:
load_dotenv()
data_dir = os.getenv("DATA_DIR")
df0 = get_db(data_dir, model)
df0

,model,A,B,Nx,dx,Nt,dt,Du,Dv,sigma_ic,random_seed,n_snapshots,filename,run_id
0,bruss,5.000000,9.000000,200,1.0,20000,0.0025,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/33b7f552-88...,NaN
1,bruss,7.102993,7.102993,400,0.5,50000,0.0010,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/4445605b-dd...,vary_ab_exp
2,bruss,3.000000,4.800000,200,1.0,20000,0.0025,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/3ceebafb-6d...,NaN
3,bruss,15.000000,30.000000,400,0.5,50000,0.0010,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/8243e11a-dd...,vary_a_b
4,bruss,2.828427,1.414214,400,0.5,50000,0.0010,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/9b605949-ea...,vary_ab_exp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,bruss,5.000000,9.000000,200,0.5,5000,0.0100,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/6972531c-43...,bruss_blowup
137,bruss,13.000000,32.500000,400,0.5,50000,0.0010,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/58e60fd9-86...,vary_a_b
138,bruss,15.000000,30.000000,200,1.0,20000,0.0025,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/1acfa38d-29...,NaN
139,bruss,5.000000,9.000000,400,0.5,50000,0.0025,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/80070ec2-69...,vary_du_dv


In [4]:
df = df0[df0['run_id'] == 'vary_sources']

In [5]:
df

,model,A,B,Nx,dx,Nt,dt,Du,Dv,sigma_ic,random_seed,n_snapshots,filename,run_id
20,bruss,5.0,9.0,200,0.5,5000,0.001,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/e2f16446-d5...,vary_sources
71,bruss,5.0,9.0,200,0.5,5000,0.001,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/2e92f34e-bf...,vary_sources
79,bruss,5.0,9.0,200,0.5,5000,0.001,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/2b7be107-20...,vary_sources
91,bruss,5.0,9.0,200,0.5,5000,0.001,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/580545ac-ad...,vary_sources
127,bruss,5.0,9.0,200,0.5,5000,0.001,2.0,22.0,0.1,1,100,/cluster/scratch/vogtva/data/bruss/6d9179d6-27...,vary_sources


In [6]:
def delete_run(df, run_id):
    for i, row in df.iterrows():
        if row['run_id'] == run_id:
            filename = row['filename']
            df.drop(i, inplace=True)
            os.remove(filename)
            os.remove(filename.replace("_output.nc", ".json"))
            os.remove(filename.replace("_output.nc", ".nc"))

    return df
